<a href="https://colab.research.google.com/github/kujayeong/TextAnalysis/blob/main/DocumentSimilarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리 설치



In [ ]:
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!pip3 install konlpy 
! JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [829 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.lau

In [ ]:
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt, Mecab
from konlpy.utils import pprint

In [ ]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
# !pip install nltk
# import nltk

In [ ]:
# ! pip install kss
# import kss

In [ ]:
import pandas as pd
import numpy as np
import re 

# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.text import text_to_word_sequence 

# 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#json 파일 불러오기
import json

file_path = '/content/drive/MyDrive/data/문서요약 텍스트/Training/'
json_file_list = ['train_original_law','train_original_news','train_original_paper']

# for json_file in json_file_list :
#   with open(file_path + json_file +'.json','r') as f:
#     globals()['data_{}'.format(json_file)] = json.load(f)

with open(file_path + 'train_original_paper.json','r') as f:
  data_train_original_paper = json.load(f)

# Json 파일 데이터프레임으로

In [ ]:
# for i in range(len(data_train_original_paper['documents'])):
for i in range(100,150):
  print(i,'요약',data_train_original_paper['documents'][i]['abstractive'][0])
  text2 = ''
  for j in range(len(data_train_original_paper['documents'][i]['text'])) :
    for k in range(len(data_train_original_paper['documents'][i]['text'][j])):
      print(j,'-',k,' :',data_train_original_paper['documents'][i]['text'][j][k]['sentence'])
  print('')

100 요약 광양시는 광양보건대가 자체적으로 자구노력을 하지 않은 상태에서 시민들의 세금으로 해당 학교의 재정지원을 하는 것은 적절하지 않다고 판단하여 보건대의 예산지원 요청을 보류한 상태이다.

0 - 0  : 학생 해외연수비 등 지원 요청…위원들“학교 노력이 우선”지적
1 - 0  : 이성훈 sinawi@hanmail.net
2 - 0  : 광양시가 광양보건대가 요청한 예산지원을 일단 보류했다.
3 - 0  : 대학 측의 학교 살리기 자구 노력 움직임을 살펴본 후 검토하겠다는 입장이다.
3 - 1  : 이에 따라 광양보건대는 대학 살리기 위한 자체 노력 없이는 시의 지원을 사실상 받을 수 없게 됐다.
4 - 0  : 시는 지난 20일, 지방대학 지원 사업 심의위원회를 열어 광양보건대가 지원을 요청한 20여억원의 지원근거 등에 대해 심의했다.
5 - 0  : 보건대는 교육부의 재정지원제한조치에도 불구하고 학생들에 대한 직접 지원은 가능함에 따라 광양시에 학생 해외연수비로 1억5000만원을 지원해 줄 것을 요청한 것으로 알려졌다.
6 - 0  : 또한 자격증 취득 과정 지원 등 20억원대 지원을 요청했다.
6 - 1  : 하지만 위원회는 특정학교 학생들을 위한 해외연수비를 시민들의 세금으로 지원하는 것이 타당한가라는 논란 끝에 결국 심의를 보류한 것으로 알려졌다.
7 - 0  : 광양보건대가 자체적으로 자구노력을 하지 않는 상태에서 교육부의 방침을 어기고 재정지원을 하는 것은 불가능하다는 의견이다.
7 - 1  : 또한 광양보건대가 지원을 요청한 일부 항목의 경우 교육부의 재정지원 제한조치에도 불구하고 지원이 가능하다고 하지만 이 역시 형평성 차원에서 당장 결정할 수는 없다는 의견이 지배적이다.
8 - 0  : 이는 지난해 11월 열린 보건대-한려대 살리기 위한 범시민대책위원회에서 나온 의견가 대부분 일치한다.
8 - 1  : 당시 한려대는 신경대와 통합하는 방안으로 자구책을 찾겠다며 지원을 호소했다.
9 - 0  : 반면 보건대는 통합 대신 신입생 유치와 시

In [ ]:
#DataFrame 만들기 함수
def json_dic2df(dictionary, label):
  Text = []
  Summary = []

  for i in range(len(dictionary['documents'])):
    Summary.append(dictionary['documents'][i]['abstractive'][0])
    text2 = ''
    for j in range(len(dictionary['documents'][i]['text'])) :
      for k in range(len(dictionary['documents'][i]['text'][j])):
        text2 += dictionary['documents'][i]['text'][j][k]['sentence'] + ' '
    Text.append(text2)
  
  df = pd.DataFrame({'Text': Text,
                       'Summary': Summary,
                       'Label' : label })

  return df

In [ ]:
df_paper = json_dic2df(data_train_original_paper, 'paper')
df_paper.head()

,Text,Summary,Label
0,ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...,paper
1,"8억 투입, 고소천사벽화·자산마을에 색채 입혀 이성훈 sinawi@hanmail.n...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...,paper
2,전남드래곤즈 해맞이 다짐…선수 영입 활발 이성훈 sinawi@hanmail.net ...,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...,paper
3,"11~24일, 매실·감·참다래 등 지역특화작목 이성훈 sinawi@hanmail.n...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작...",paper
4,"홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항 이성훈 sinawi@hanma...",올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...,paper


# Preprocessing

## 중복 제거

In [ ]:
len(df_paper)

271093

In [ ]:
print('Text 열에서 중복을 배제한 유일한 샘플의 수:', df_paper['Text'].nunique())
print('Summary 열에서 중복을 배제한 유일한 샘플의 수 :', df_paper['Summary'].nunique())

Text 열에서 중복을 배제한 유일한 샘플의 수: 270637
Summary 열에서 중복을 배제한 유일한 샘플의 수 : 271076


In [ ]:
df_paper.drop_duplicates(subset=['Text'], inplace=True)

In [ ]:
len(df_paper)

270637

In [ ]:
df_paper.isnull().sum()

Text       0
Summary    0
Label      0
dtype: int64

## 불용어


In [ ]:
#한국어 불용어 리스트
korean_stopwords = pd.read_excel('/content/drive/MyDrive/korean_stopwords.xlsx', header=None)
korean_stopwords = list(korean_stopwords[0])
korean_stopwords.extend(['된다','한다','고', '했다','이다','는','이','가','은', '에서', '으로', '에는'])

print(korean_stopwords)

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해', '을', '를', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면', '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '비길수 없다', '해서는 안된다', '뿐만 아니라', '만이 아니다', '만은 아니다', '막론하고', '관계없이', '그치지 않다', '그러나', '그런데', '하지만', '든간에', '논하지 않다', '따지지 않다', '설사', '비록', '더라도', '아니면', '만 못하다', '하는 편이 낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다', '오르다', '제외하고', '이 외에', '이 밖에', '하여야', '비로소', '한다면 몰라도', '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할 생각이다', '하려고하다', '이리하여', '그리하여', '그렇게 함으로써', '하지만', '일때', '할때', '앞에서', '중에서', '보는데서', '으로써', '로써', '까지', '해야한다', '일것이다', '반드시', '할줄알다', '할수있다', '할수있어', '임에 틀림없다', '한다면', '등', '등등', '제', '겨우', '단지', '다만', '할뿐', '딩동', '댕그', '대해서', '대하여', '대하면', '훨씬', '얼마나', '얼마만큼', '얼마큼', '남짓', '여', '얼마간', '약간', '다소', '좀', '조금', '다수', '몇', '얼마', '지만', '하물며', '또한', '그러나', '그렇지만', '하지만', '이외에도', '대해 말하자면', '뿐이다', '다음에', '반대로', '반대로 말하자면', '이와 반대로',

## 형태소 분석기
1. jhannanum(한나눔) 
2. kkma(꼬꼬마)
3. komoran(코모란)
4. twitter(트위터) -> 현 Okt
5. mecab(메캅)

In [ ]:
df_paper['Text'][200]

'지역별 시? 도의원 출마, 체면 대신‘현실’선택 이성훈 sinawi@hanmail.net 민주평화당이 우리 지역에서 시장 후보를 내지 않기로 결론을 내렸다. 여러 후보군을 물색했지만 현실적으로 지지율이 높지 않은데다 당선 가능성이 희박한 상황에서 후보를 냈다가 망신을 당할 경우 자칫 당의 지지율마저 대폭 떨어질 것이라는 염려 때문이다. 이는 정인화 국회의원이 소속된 민평당에서 시장 후보를 내지 않을 경우 체면을 구긴다는 우려도 있지만 체면만 중시한 채 후보를 공천했다가 선거비용을 보전 받는 기준인‘득표율 10% 이상’을 넘지 못할 경우 찾아오는 충격이 더욱더 크다는 현실적인 이유 때문인 것으로 풀이된다. 민평당은 이런 가운데 시도의원 후보를 지역별로 내세워 선거를 치르되 지방선거에서 당선을 목표로 사활을 내걸고 실속을 다진다는 계획이다. 민평당은 그동안 남기호·이정문 전 광양시의회 의장의 출마설이 꾸준히 제기되어 왔다. 특히 최근에는 남기호 전 의장이 민평당 소속으로 시장 출마를 할 계획이었으나 남 전 의장에게 직접 확인한 결과 도의원 출마를 결정했다. <관련기사 4면> 남 전 의장은“선거는 이기기 위해 출마하는 것인데 현실적으로 우리당 소속으로 출마하는 것은 당선 보장이 어렵다”고 밝혔다. 시장 예비후보는 현재 김재무·김현옥·이옥재 예비후보가 등록한 상태인데 여기에 무소속으로 정현복 현 시장이 출마 선언을 할 것으로 보인다. 남 전 의장은“당선도 현실적으로 어려운데다 출마했다가 만일 득표율이 형편없이 낮게 나올 경우 제 개인적인 정치 생활은 물론, 당에 미치는 영향도 상당하다”며“민평당이 국회의원을 보유하고 있지만 현실적인 정치여건을 감안하면 이번 지방선거에서 시장 출마를 할 수 없는 상황이다”고 설명했다. 민평당 지역위원회 관계자 역시“여러 여건을 봤을 때 시장 후보는 배출하기 어렵다”며“지역별로 시도의원 후보를 내세워 승부를 거는 것으로 당내 가닥을 잡아가고 있다”고 밝혔다. 이 관계자는 “도의원은 1선거구와 3선거구, 시의원은 가나다라 각 선거구에 후

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

print('Mecab')

text = df_paper['Text'][200]
print('원문장:\n', text )

text = mecab.morphs(text) #형태소
print('형태소:\n', text)

p = re.compile('[가-힣]+')
text = p.findall((' ').join(text)) #한글 외에 제거
print('특수문자 제거:\n', text)

text2 = [word for word in text if not word in korean_stopwords] #불용어 제거
text2 = [word for word in text2 if len(word) >1 ] #한글자 제거
print('불용어 제거:\n', (' ').join(text2))

Mecab
원문장:
 지역별 시? 도의원 출마, 체면 대신‘현실’선택 이성훈 sinawi@hanmail.net 민주평화당이 우리 지역에서 시장 후보를 내지 않기로 결론을 내렸다. 여러 후보군을 물색했지만 현실적으로 지지율이 높지 않은데다 당선 가능성이 희박한 상황에서 후보를 냈다가 망신을 당할 경우 자칫 당의 지지율마저 대폭 떨어질 것이라는 염려 때문이다. 이는 정인화 국회의원이 소속된 민평당에서 시장 후보를 내지 않을 경우 체면을 구긴다는 우려도 있지만 체면만 중시한 채 후보를 공천했다가 선거비용을 보전 받는 기준인‘득표율 10% 이상’을 넘지 못할 경우 찾아오는 충격이 더욱더 크다는 현실적인 이유 때문인 것으로 풀이된다. 민평당은 이런 가운데 시도의원 후보를 지역별로 내세워 선거를 치르되 지방선거에서 당선을 목표로 사활을 내걸고 실속을 다진다는 계획이다. 민평당은 그동안 남기호·이정문 전 광양시의회 의장의 출마설이 꾸준히 제기되어 왔다. 특히 최근에는 남기호 전 의장이 민평당 소속으로 시장 출마를 할 계획이었으나 남 전 의장에게 직접 확인한 결과 도의원 출마를 결정했다. <관련기사 4면> 남 전 의장은“선거는 이기기 위해 출마하는 것인데 현실적으로 우리당 소속으로 출마하는 것은 당선 보장이 어렵다”고 밝혔다. 시장 예비후보는 현재 김재무·김현옥·이옥재 예비후보가 등록한 상태인데 여기에 무소속으로 정현복 현 시장이 출마 선언을 할 것으로 보인다. 남 전 의장은“당선도 현실적으로 어려운데다 출마했다가 만일 득표율이 형편없이 낮게 나올 경우 제 개인적인 정치 생활은 물론, 당에 미치는 영향도 상당하다”며“민평당이 국회의원을 보유하고 있지만 현실적인 정치여건을 감안하면 이번 지방선거에서 시장 출마를 할 수 없는 상황이다”고 설명했다. 민평당 지역위원회 관계자 역시“여러 여건을 봤을 때 시장 후보는 배출하기 어렵다”며“지역별로 시도의원 후보를 내세워 승부를 거는 것으로 당내 가닥을 잡아가고 있다”고 밝혔다. 이 관계자는 “도의원은 1선거구와 3선거구, 시의원은 가나

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

def preprocessing(sentence) :
  morphs_list = mecab.morphs(sentence) #형태소
  
  p = re.compile('[가-힣]+')
  cleaned_list = p.findall((' ').join(morphs_list)) #특수문자 제거
  
  word_list = [word for word in cleaned_list if not word in korean_stopwords] #불용어 제거
  word_list2 = [word for word in word_list if len(word) >1 ] #한글자 제거
  return word_list2

## 전처리 후 pickle 파일로 저장

In [ ]:
# preprocessed_paper = []

# for sentence in df_paper['Text'][:100000]:
#   preprocessed_sen = preprocessing(sentence)
#   preprocessed_paper.append(preprocessed_sen)

In [ ]:
# import pickle

# #dataframe 피클로 저장
# with open( "/content/drive/MyDrive/data/문서요약 텍스트/Training/preprocessed_paper1.pkl", "wb" ) as file:
#     pickle.dump(preprocessed_paper, file)

In [ ]:
# preprocessed_paper = []

# for sentence in df_paper['Text'][100000:200000]:
#   preprocessed_sen = preprocessing(sentence)
#   preprocessed_paper.append(preprocessed_sen)

In [ ]:
# import pickle

# with open( "/content/drive/MyDrive/data/문서요약 텍스트/Training/preprocessed_paper2.pkl", "wb" ) as file:
#     pickle.dump(preprocessed_paper, file)

In [ ]:
# for sentence in df_paper['Text'][200000:]:
#   preprocessed_sen = preprocessing(sentence)
#   preprocessed_paper.append(preprocessed_sen)

In [ ]:
# import pickle

# with open( "/content/drive/MyDrive/data/문서요약 텍스트/Training/preprocessed_paper3.pkl", "wb" ) as file:
#     pickle.dump(preprocessed_paper, file)

10만 행 15분

In [ ]:
# preprocessed_paper_s = []

# for sentence in df_paper['Summary']:
#   preprocessed_sen = preprocessing(sentence)
#   preprocessed_paper_s.append(preprocessed_sen)

# preprocessed_paper_s[:5]

# import pickle

# with open( "preprocessed_paper4.pkl", "wb" ) as file:
#   pickle.dump(preprocessed_paper_s, file)

## pickle 파일 불러오기

In [ ]:
#pickle 파일 불러와서 df 만들기
import pickle

#pickle 파일 불러오기
with open( "/content/drive/MyDrive/data/문서요약 텍스트/Training/preprocessed_paper1.pkl", "rb" ) as file:
  preprocessed_paper1 = pickle.load(file)

with open( "/content/drive/MyDrive/data/문서요약 텍스트/Training/preprocessed_paper3.pkl", "rb" ) as file:
  preprocessed_paper3 = pickle.load(file)

preprocessed_paper = preprocessed_paper1 + preprocessed_paper3

In [ ]:
import sys
print('preprocessed_paper 데이터 타입:',type(preprocessed_paper))
print('preprocessed_paper 길이:', len(preprocessed_paper))
print('preprocessed_paper 데이터 크기:',sys.getsizeof(preprocessed_paper))
print()
print(df_paper['Text'][0])
print(preprocessed_paper[0])

preprocessed_paper 데이터 타입: <class 'list'>
preprocessed_paper 길이: 270637
preprocessed_paper 데이터 크기: 2165168

ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다. 농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다. 최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다. 
['조

# 정수 인코딩

In [ ]:
#단어집합
vocab = {} #단어, 빈도수

for sentence in preprocessed_paper :
  for word in sentence :
    if word not in vocab :
      vocab[word] = 0
    vocab[word] += 1

#빈도수 높은 순으로 정렬
vocab_sorted = sorted(vocab.items(), key = lambda x:x[1], reverse = True) 


#단어_인덱스 부여
word_to_index = {}

for (word, frequency) in vocab_sorted :
  if frequency > 1 : #빈도수 1인 단어는 제외
    word_to_index[word] = len(word_to_index) + 1

#oov(Out-of-Vocabulary) 단어 집합에 없는 단어
word_to_index['OOV'] = len(word_to_index) + 1

word_to_index

{'지역': 1,
 '사업': 2,
 '지원': 3,
 '위해': 4,
 '통해': 5,
 '기자': 6,
 '지난': 7,
 '대한': 8,
 '기업': 9,
 '면서': 10,
 '밝혔': 11,
 '계획': 12,
 '한국': 13,
 '위한': 14,
 '시장': 15,
 '진행': 16,
 '교육': 17,
 '도록': 18,
 '추진': 19,
 '운영': 20,
 '다는': 21,
 '다고': 22,
 '대표': 23,
 '올해': 24,
 '산업': 25,
 '관련': 26,
 '문화': 27,
 '개발': 28,
 '예정': 29,
 '도시': 30,
 '지난해': 31,
 '센터': 32,
 '경제': 33,
 '가능': 34,
 '정부': 35,
 '제공': 36,
 '사회': 37,
 '대상': 38,
 '참여': 39,
 '관계자': 40,
 '기술': 41,
 '보다': 42,
 '다양': 43,
 '의원': 44,
 '시민': 45,
 '라고': 46,
 '시설': 47,
 '기관': 48,
 '이날': 49,
 '주민': 50,
 '서울': 51,
 '관리': 52,
 '조사': 53,
 '활동': 54,
 '경우': 55,
 '서비스': 56,
 '정책': 57,
 '전국': 58,
 '라는': 59,
 '특히': 60,
 '대해': 61,
 '학교': 62,
 '환경': 63,
 '경기': 64,
 '현재': 65,
 '따르': 66,
 '행사': 67,
 '필요': 68,
 '마련': 69,
 '라며': 70,
 '선정': 71,
 '광주': 72,
 '대회': 73,
 '안전': 74,
 '투자': 75,
 '위원회': 76,
 '확대': 77,
 '평가': 78,
 '결과': 79,
 '학생': 80,
 '조성': 81,
 '최근': 82,
 '문제': 83,
 '이라고': 84,
 '국내': 85,
 '규모': 86,
 '이후': 87,
 '기준': 88,
 '해야': 89,
 '시작': 90,
 '국가': 91,
 '세

In [ ]:
#index_to_word
index_to_word = {}

for word in word_to_index.keys() :
  index = word_to_index[word]
  index_to_word[index] = word

index_to_word

{1: '지역',
 2: '사업',
 3: '지원',
 4: '위해',
 5: '통해',
 6: '기자',
 7: '지난',
 8: '대한',
 9: '기업',
 10: '면서',
 11: '밝혔',
 12: '계획',
 13: '한국',
 14: '위한',
 15: '시장',
 16: '진행',
 17: '교육',
 18: '도록',
 19: '추진',
 20: '운영',
 21: '다는',
 22: '다고',
 23: '대표',
 24: '올해',
 25: '산업',
 26: '관련',
 27: '문화',
 28: '개발',
 29: '예정',
 30: '도시',
 31: '지난해',
 32: '센터',
 33: '경제',
 34: '가능',
 35: '정부',
 36: '제공',
 37: '사회',
 38: '대상',
 39: '참여',
 40: '관계자',
 41: '기술',
 42: '보다',
 43: '다양',
 44: '의원',
 45: '시민',
 46: '라고',
 47: '시설',
 48: '기관',
 49: '이날',
 50: '주민',
 51: '서울',
 52: '관리',
 53: '조사',
 54: '활동',
 55: '경우',
 56: '서비스',
 57: '정책',
 58: '전국',
 59: '라는',
 60: '특히',
 61: '대해',
 62: '학교',
 63: '환경',
 64: '경기',
 65: '현재',
 66: '따르',
 67: '행사',
 68: '필요',
 69: '마련',
 70: '라며',
 71: '선정',
 72: '광주',
 73: '대회',
 74: '안전',
 75: '투자',
 76: '위원회',
 77: '확대',
 78: '평가',
 79: '결과',
 80: '학생',
 81: '조성',
 82: '최근',
 83: '문제',
 84: '이라고',
 85: '국내',
 86: '규모',
 87: '이후',
 88: '기준',
 89: '해야',
 90: '시작',
 91: '국가',
 92

In [ ]:
import pickle

with open( "/content/drive/MyDrive/data/문서요약 텍스트/Training/vocab_sorted.pkl", "wb" ) as file:
  pickle.dump(vocab_sorted, file)

with open( "/content/drive/MyDrive/data/문서요약 텍스트/Training/word_to_index.pkl", "wb" ) as file:
  pickle.dump(word_to_index, file)

with open( "/content/drive/MyDrive/data/문서요약 텍스트/Training/index_to_word.pkl", "wb" ) as file:
  pickle.dump(index_to_word, file)

In [ ]:
encoded_text = []
for document in preprocessed_paper :
  encoded = []
  for word in  document :
    try:
      encoded.append(word_to_index[word])
    except KeyError :
      encoded.append(word_to_index['OOV'])
  encoded_text.append(encoded)

print(encoded_text[0])

[13571, 2721, 4969, 3, 11328, 4016, 4311, 83, 2605, 337, 4, 24, 213, 189, 442, 136, 19, 1171, 189, 442, 1005, 3409, 2721, 195, 2721, 522, 521, 513, 1408, 227, 24, 176, 2721, 1217, 12, 734, 58, 1224, 213, 2490, 562, 1841, 1810, 3, 2, 38, 1858, 6308, 3539, 1341, 2721, 1217, 276, 3, 38, 2721, 179, 35007, 273, 2721, 182, 1217, 734, 77, 1271, 4311, 375, 3898, 4556, 3167, 11134, 1271, 2012, 890, 543, 3539, 55, 703, 2721, 1217, 919, 1047, 15638, 35, 1068, 7541, 3539, 8794, 785, 2, 1763, 1408, 6308, 3638, 3539, 543, 29, 3, 2983, 283, 3409, 2721, 330, 2721, 7994, 2721, 513, 2421, 73404, 1455, 2721, 4969, 3, 2721, 1217, 2440, 445, 3539, 5972, 16814, 192, 89, 176, 1437, 1841, 26, 48, 445, 39, 2721, 3, 2, 19, 523, 149, 1, 709, 5695, 71, 2, 747, 392, 19, 367, 83274, 4016, 661, 276, 685, 82, 13990, 268, 1331, 954, 579, 261, 4311, 288, 185, 518, 445, 687, 261, 4311, 444, 337, 18, 2721, 1217, 3, 2, 491, 39, 526]


In [ ]:
print(df_paper['Text'][0])
print(preprocessed_paper[0])
print(encoded_text[0])
print()
print('13571 : ', index_to_word[13571])
print('2721 : ', index_to_word[2721])
print('4969 : ', index_to_word[4969])

ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다. 농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다. 최향철 전라남도 친환경농업과장은 “최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다. 
['조사료', '작물', '차등', '지원', '이성훈', '전라남도', '과잉', '문제', '근본', '해결', '위해', '올해', '시행', '생산', '조정', '적극', '추진', '키로'

In [ ]:
import pickle

#dataframe 피클로 저장
with open( "/content/drive/MyDrive/data/문서요약 텍스트/Training/encoded_text.pkl", "wb" ) as file:
    pickle.dump( encoded_text, file)

In [ ]:
import pickle

#pickle 파일 불러오기
with open( "/content/drive/MyDrive/data/문서요약 텍스트/Training/encoded_text.pkl", "rb" ) as file:
    encoded_text = pickle.load(file)

print(encoded_text[0])

[13571, 2721, 4969, 3, 11328, 4016, 4311, 83, 2605, 337, 4, 24, 213, 189, 442, 136, 19, 1171, 189, 442, 1005, 3409, 2721, 195, 2721, 522, 521, 513, 1408, 227, 24, 176, 2721, 1217, 12, 734, 58, 1224, 213, 2490, 562, 1841, 1810, 3, 2, 38, 1858, 6308, 3539, 1341, 2721, 1217, 276, 3, 38, 2721, 179, 35007, 273, 2721, 182, 1217, 734, 77, 1271, 4311, 375, 3898, 4556, 3167, 11134, 1271, 2012, 890, 543, 3539, 55, 703, 2721, 1217, 919, 1047, 15638, 35, 1068, 7541, 3539, 8794, 785, 2, 1763, 1408, 6308, 3638, 3539, 543, 29, 3, 2983, 283, 3409, 2721, 330, 2721, 7994, 2721, 513, 2421, 73404, 1455, 2721, 4969, 3, 2721, 1217, 2440, 445, 3539, 5972, 16814, 192, 89, 176, 1437, 1841, 26, 48, 445, 39, 2721, 3, 2, 19, 523, 149, 1, 709, 5695, 71, 2, 747, 392, 19, 367, 83274, 4016, 661, 276, 685, 82, 13990, 268, 1331, 954, 579, 261, 4311, 288, 185, 518, 445, 687, 261, 4311, 444, 337, 18, 2721, 1217, 3, 2, 491, 39, 526]


In [ ]:
import sys
print('데이터 타입:',type(encoded_text))
print('길이:', len(encoded_text))
print('데이터 크기:',sys.getsizeof(encoded_text))
print()
print(encoded_text[0])

데이터 타입: <class 'list'>
길이: 270637
데이터 크기: 2241120

[13571, 2721, 4969, 3, 11328, 4016, 4311, 83, 2605, 337, 4, 24, 213, 189, 442, 136, 19, 1171, 189, 442, 1005, 3409, 2721, 195, 2721, 522, 521, 513, 1408, 227, 24, 176, 2721, 1217, 12, 734, 58, 1224, 213, 2490, 562, 1841, 1810, 3, 2, 38, 1858, 6308, 3539, 1341, 2721, 1217, 276, 3, 38, 2721, 179, 35007, 273, 2721, 182, 1217, 734, 77, 1271, 4311, 375, 3898, 4556, 3167, 11134, 1271, 2012, 890, 543, 3539, 55, 703, 2721, 1217, 919, 1047, 15638, 35, 1068, 7541, 3539, 8794, 785, 2, 1763, 1408, 6308, 3638, 3539, 543, 29, 3, 2983, 283, 3409, 2721, 330, 2721, 7994, 2721, 513, 2421, 73404, 1455, 2721, 4969, 3, 2721, 1217, 2440, 445, 3539, 5972, 16814, 192, 89, 176, 1437, 1841, 26, 48, 445, 39, 2721, 3, 2, 19, 523, 149, 1, 709, 5695, 71, 2, 747, 392, 19, 367, 83274, 4016, 661, 276, 685, 82, 13990, 268, 1331, 954, 579, 261, 4311, 288, 185, 518, 445, 687, 261, 4311, 444, 337, 18, 2721, 1217, 3, 2, 491, 39, 526]


# np.array로 변환

In [ ]:
encoded_text_np = np.array(encoded_text)

import sys
print('데이터 타입:',type(encoded_text_np))
print('길이:', len(encoded_text_np))
print('데이터 크기:',sys.getsizeof(encoded_text_np))
print()
print(encoded_text_np[0])

데이터 타입: <class 'numpy.ndarray'>
길이: 270637
데이터 크기: 2165192

[13571, 2721, 4969, 3, 11328, 4016, 4311, 83, 2605, 337, 4, 24, 213, 189, 442, 136, 19, 1171, 189, 442, 1005, 3409, 2721, 195, 2721, 522, 521, 513, 1408, 227, 24, 176, 2721, 1217, 12, 734, 58, 1224, 213, 2490, 562, 1841, 1810, 3, 2, 38, 1858, 6308, 3539, 1341, 2721, 1217, 276, 3, 38, 2721, 179, 35007, 273, 2721, 182, 1217, 734, 77, 1271, 4311, 375, 3898, 4556, 3167, 11134, 1271, 2012, 890, 543, 3539, 55, 703, 2721, 1217, 919, 1047, 15638, 35, 1068, 7541, 3539, 8794, 785, 2, 1763, 1408, 6308, 3638, 3539, 543, 29, 3, 2983, 283, 3409, 2721, 330, 2721, 7994, 2721, 513, 2421, 73404, 1455, 2721, 4969, 3, 2721, 1217, 2440, 445, 3539, 5972, 16814, 192, 89, 176, 1437, 1841, 26, 48, 445, 39, 2721, 3, 2, 19, 523, 149, 1, 709, 5695, 71, 2, 747, 392, 19, 367, 83274, 4016, 661, 276, 685, 82, 13990, 268, 1331, 954, 579, 261, 4311, 288, 185, 518, 445, 687, 261, 4311, 444, 337, 18, 2721, 1217, 3, 2, 491, 39, 526]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [ ]:
a = sys.getsizeof(encoded_text)
b = sys.getsizeof(encoded_text_np)

print(a > b)
print(a-b)

True
75928


In [ ]:
import pickle

#dataframe 피클로 저장
with open( "/content/drive/MyDrive/data/문서요약 텍스트/Training/encoded_text_np.pkl", "wb" ) as file:
    pickle.dump( encoded_text_np, file)

In [ ]:
import pickle

#pickle 파일 불러오기
with open( "/content/drive/MyDrive/data/문서요약 텍스트/Training/encoded_text_np.pkl", "rb" ) as file:
    encoded_text_np = pickle.load(file)

print(encoded_text_np[0])

[13571, 2721, 4969, 3, 11328, 4016, 4311, 83, 2605, 337, 4, 24, 213, 189, 442, 136, 19, 1171, 189, 442, 1005, 3409, 2721, 195, 2721, 522, 521, 513, 1408, 227, 24, 176, 2721, 1217, 12, 734, 58, 1224, 213, 2490, 562, 1841, 1810, 3, 2, 38, 1858, 6308, 3539, 1341, 2721, 1217, 276, 3, 38, 2721, 179, 35007, 273, 2721, 182, 1217, 734, 77, 1271, 4311, 375, 3898, 4556, 3167, 11134, 1271, 2012, 890, 543, 3539, 55, 703, 2721, 1217, 919, 1047, 15638, 35, 1068, 7541, 3539, 8794, 785, 2, 1763, 1408, 6308, 3638, 3539, 543, 29, 3, 2983, 283, 3409, 2721, 330, 2721, 7994, 2721, 513, 2421, 73404, 1455, 2721, 4969, 3, 2721, 1217, 2440, 445, 3539, 5972, 16814, 192, 89, 176, 1437, 1841, 26, 48, 445, 39, 2721, 3, 2, 19, 523, 149, 1, 709, 5695, 71, 2, 747, 392, 19, 367, 83274, 4016, 661, 276, 685, 82, 13990, 268, 1331, 954, 579, 261, 4311, 288, 185, 518, 445, 687, 261, 4311, 444, 337, 18, 2721, 1217, 3, 2, 491, 39, 526]


# TF-IDF

## TF-IDF 에 단어 몇개를 사용할지?

In [ ]:
#전체 단어집합에서 단어 빈도수 비중

#단어 전체 빈도수 합
sum_freq = 0
for word, freq in vocab_sorted :
  sum_freq += freq

#단어별 빈도수 비중
vocab_importance = {}
for word,freq in vocab_sorted :
  vocab_importance[word] = freq/sum_freq
vocab_importance_sorted = sorted(vocab_importance.items(), key = lambda x:x[1], reverse = True) #빈도수 높은 순으로 정렬

vocab_importance_sorted

[('지역', 0.005439838299033033),
 ('사업', 0.005425243025063422),
 ('지원', 0.003892812601339516),
 ('위해', 0.00363409486803397),
 ('통해', 0.0033732501852554654),
 ('기자', 0.0032481268817772906),
 ('지난', 0.003121170001167255),
 ('대한', 0.0029890424330453725),
 ('기업', 0.0028883423769635822),
 ('면서', 0.002866467801780484),
 ('밝혔', 0.0027428296957791166),
 ('계획', 0.002671448538035779),
 ('한국', 0.0026238305399213565),
 ('위한', 0.002573443840337824),
 ('시장', 0.0025139259266376253),
 ('진행', 0.002444139980999007),
 ('교육', 0.0024281878599518186),
 ('도록', 0.002309885463404166),
 ('추진', 0.0023047147758923185),
 ('운영', 0.0022813366674610943),
 ('다는', 0.002264596108247206),
 ('다고', 0.002181498392631279),
 ('대표', 0.002152564545490517),
 ('올해', 0.0020990057574688656),
 ('산업', 0.0020174299108723826),
 ('관련', 0.0019935384008442376),
 ('문화', 0.0019498442577919968),
 ('개발', 0.0019468371912957454),
 ('예정', 0.0019333970709192064),
 ('도시', 0.0019061684505110748),
 ('지난해', 0.0018838721525876487),
 ('센터', 0.00188003997

In [ ]:
#인덱스 1번부터 index번까지가 전체에서 차지하는 비중
index = 10000

importance = 0
for i in range(0,index) :
  #0부터 9999까지
  importance += vocab_importance_sorted[i][1]

print('index {}번까지의 단어의 전체 비중은 {}% 이다.'.format(index, round(importance*100, 4)))

index 10000번까지의 단어의 전체 비중은 89.5409% 이다.


In [ ]:
x, y = vocab_importance_sorted[0],  vocab_importance_sorted[index-1]
print(x, y)
print('{}의 index : {}\n {}의 index : {} '.format(x[0], word_to_index[x[0]],y[0] ,word_to_index[y[0]]))

('지역', 0.005439838299033033) ('컬링', 9.33290760117101e-06)
지역의 index : 1
 컬링의 index : 10000 


index 10000까지 활용하기

## TF

In [ ]:
len(encoded_text_np)

270637

In [ ]:
import time
start = time.time()  # 시작 시간 저장


#index는 1~10000
#tf행은 0~9999
tf_list = np.zeros(10000)

for document in encoded_text_np[:10000] :
  tf = np.zeros(10000)

  for k in document :
    if k <= 10000 :
      tf[k-1] += 1
    else :
      continue
  tf_list = np.vstack([tf_list,tf])

tf_list = np.delete(tf_list,(0), axis = 0)

print(tf_list.shape)

print("time :", time.time() - start) 

(10000, 10000)
time : 805.1688935756683


In [ ]:
import pickle

with open( "/content/drive/MyDrive/data/문서요약 텍스트/Training/tf_list(~10000).pkl", "wb" ) as file:
    pickle.dump( tf_list, file)

In [ ]:
import time
start = time.time()  # 시작 시간 저장

tf_list = np.zeros(10000)

for document in encoded_text_np[10000:20000] :
  tf = np.zeros(10000)

  count = {}
  for index in document : 
    try : count[index] += 1
    except :  count[index] = 1
  
  for index in count.keys() :
    try : tf[index-1] = count[index]
    except : continue
  tf_list = np.vstack([tf_list,tf])

tf_list = np.delete(tf_list,(0), axis = 0)

print(tf_list.shape)
print("time :", time.time() - start) 

(10000, 10000)
time : 820.1468510627747


In [ ]:
import pickle

#pickle 저장하기
with open( "/content/drive/MyDrive/data/문서요약 텍스트/Training/tf_list(10000~20000).pkl", "wb" ) as file:
    pickle.dump( tf_list, file)

In [ ]:
import time
start = time.time()  # 시작 시간 저장

tf_list = np.zeros(10000)

for document in encoded_text_np[80000:100000] :
  tf = np.zeros(10000)

  count = {}
  for index in document : 
    try : count[index] += 1
    except :  count[index] = 1
  
  for index in count.keys() :
    try : tf[index-1] = count[index]
    except : continue
  tf_list = np.vstack([tf_list,tf])

tf_list = np.delete(tf_list,(0), axis = 0)

print(tf_list.shape)
print("time :", time.time() - start) 

(20000, 10000)
time : 3042.7013540267944


In [ ]:
# import pickle

# #pickle 저장하기
# with open( "/content/drive/MyDrive/data/문서요약 텍스트/Training/tf_list(80000~100000).pkl", "wb" ) as file:
#     pickle.dump( tf_list, file)

In [ ]:
tf_list.shape

(20000, 10000)

(44788, 10000) 만드는데 4시간 걸림

In [ ]:
import sys
print('데이터 타입:',type(tf_list))
print('shape:', tf_list.shape)
print('메모리 크기:',sys.getsizeof(tf_list))

데이터 타입: <class 'numpy.ndarray'>
shape: (20000, 10000)
메모리 크기: 1600000112


In [ ]:
import pickle

#pickle 불러오기
with open( "/content/drive/MyDrive/data/문서요약 텍스트/Training/tf_list.pkl", "rb" ) as file:
    tf_list = pickle.load(file)

print(tf_list[0])

## IDF

encoded_text_np[60000:80000] 로 만든 tf_list로 만든 df 와 idf

In [ ]:
import time
start = time.time()  # 시작 시간 저장

new_tf_list = np.zeros(10000)

for tf in tf_list[:1000]:
  tf_new = [1 if i >1 else i for i in tf]
  new_tf_list  = np.vstack([new_tf_list ,tf_new])

new_tf_list = np.delete(new_tf_list,(0), axis = 0)

print()
print(new_tf_list.shape)
print("time :", time.time() - start) 



(1000, 10000)
time : 14.697517395019531


In [ ]:
import time
start = time.time()  # 시작 시간 저장

new_tf_list2 = np.zeros(10000)

for tf in tf_list[1000:2000]:
  tf[tf > 1] = 1
  new_tf_list2  = np.vstack([new_tf_list2 ,tf_new])

new_tf_list2 = np.delete(new_tf_list2,(0), axis = 0)

print()
print(new_tf_list2.shape)
print("time :", time.time() - start) 


(1000, 10000)
time : 10.744094848632812


In [ ]:
import time
start = time.time()  # 시작 시간 저장

new_tf_list3 = np.zeros(10000)

for tf in tf_list[2000:3000]:
  np.where(tf > 1, 1, tf)
  new_tf_list3  = np.vstack([new_tf_list3 ,tf_new])

new_tf_list3 = np.delete(new_tf_list3,(0), axis = 0)

print()
print(new_tf_list3.shape)
print("time :", time.time() - start) 


(1000, 10000)
time : 10.840654611587524


In [ ]:
import time
start = time.time()  # 시작 시간 저장

new_tf_list4 = np.zeros(10000)

for tf in tf_list[3000:4000]:
  np.clip(tf, 0, 1, out=tf)
  new_tf_list4  = np.vstack([new_tf_list4 ,tf_new])

new_tf_list4 = np.delete(new_tf_list4,(0), axis = 0)

print()
print(new_tf_list4.shape)
print("time :", time.time() - start) 


(1000, 10000)
time : 10.897003650665283


In [ ]:
import time
start = time.time()  # 시작 시간 저장

new_tf_list = np.zeros(10000)

for tf in tf_list:
  tf[tf > 1] = 1
  new_tf_list  = np.vstack([new_tf_list ,tf_new])

new_tf_list = np.delete(new_tf_list,(0), axis = 0)

print()
print(new_tf_list.shape)
print("time :", time.time() - start) 


(19999, 10000)
time : 3301.076908349991


In [ ]:
import pickle

#pickle 저장하기
with open( "/content/drive/MyDrive/data/문서요약 텍스트/Training/boolean_tf_list.pkl", "wb" ) as file:
    pickle.dump(new_tf_list, file)

In [ ]:
#df 열 합계
df = new_tf_list.sum(axis=0)

In [ ]:
#idf
from math import log 
N = len(tf_list) #전체 문서 개수
#log(N/(i+1))+1

idf = np.log1p(np.reciprocal(df+1)) + 1

In [ ]:
idf

array([1.69314718, 1.69314718, 1.69314718, ..., 1.69314718, 1.69314718,
       1.69314718])

In [ ]:
tf_idf = tf_list*idf

In [ ]:
tf_idf.shape

(19999, 10000)

In [ ]:
import pickle

#pickle 저장하기
with open( "/content/drive/MyDrive/data/문서요약 텍스트/Training/tf_idf.pkl", "wb" ) as file:
    pickle.dump(tf_idf, file)

# 유사도 계산 함수

In [ ]:
#코사인 유사도
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
a=100
cos_sim_dic = {}

for i in range(0,100) :
  key = i
  value = cos_sim(tf_idf[a],tf_idf[i])
  cos_sim_dic[key] = value

#유사도 높은 순으로 정렬
cos_sim_dic_sorted = sorted(cos_sim_dic.items(), key = lambda x:x[1], reverse = True) 

print(cos_sim_dic_sorted[:5])

[(26, 0.21987198162791313), (24, 0.2015193305200968), (34, 0.19603861580866758), (20, 0.19085017946052332), (43, 0.17925308468524856)]


In [ ]:
now_encoded_text_np = encoded_text_np[60000:80000]
print(len(now_encoded_text_np))

20000


In [ ]:
print(now_encoded_text_np[100])
print(now_encoded_text_np[26])

[591, 3100, 547, 1882, 89, 66724, 103, 1365, 19, 591, 3100, 547, 1882, 89, 66724, 103, 1365, 19, 19934, 6, 159, 117, 951, 1668, 17, 32, 202, 1450, 449, 1365, 922, 130, 996, 9392, 6, 1450, 449, 1828, 779, 5, 117, 1450, 1873, 1033, 30, 110, 21, 786, 367, 870, 2370, 3033, 310, 4069, 10, 117, 1450, 449, 1955, 547, 2, 5725, 591, 3100, 117, 260, 786, 17702, 117, 951, 1668, 17, 32, 1450, 449, 1365, 59, 350, 922, 591, 3100, 1, 1274, 5487, 794, 3097, 399, 143, 70, 66724, 2267, 1955, 103, 749, 2857, 1041, 14240, 310, 2965, 49, 4393, 119598, 591, 3100, 117, 260, 1015, 23507, 1043, 786, 14215, 817, 764, 9, 348, 8068, 1, 511, 4828, 822, 94, 1450, 449, 1955, 2370, 3033, 267, 149451, 117, 4988, 197, 1464, 612, 764, 45200, 2595, 1450, 449, 59, 536, 24688, 70, 2493, 870, 1114, 807, 3864, 441, 7079, 1575, 1837, 716, 471, 6526, 1639, 870, 25851, 1016, 9305, 84, 247, 159, 117, 951, 1668, 17, 32, 202, 1450, 449, 1365, 922, 130, 996, 9392, 6, 60, 922, 1450, 449, 1955, 872, 409, 114, 1736, 481, 2855, 114, 29

In [ ]:
tf_100 = np.zeros(10000)
count = {}
for index in now_encoded_text_np[101] : 
  try : count[index] += 1
  except :  count[index] = 1

for index in count.keys() :
  try : tf_100[index-1] = count[index]
  except : continue


tf_26 = np.zeros(10000)
count = {}
for index in now_encoded_text_np[27] : 
  try : count[index] += 1
  except :  count[index] = 1

for index in count.keys() :
  try : tf_26[index-1] = count[index]
  except : continue

tf_idf_100 = tf_100 * idf
tf_idf_26 = tf_26 * idf

print('코사인 유사도 : ', cos_sim(tf_idf_100,tf_idf_26))

코사인 유사도 :  0.3375974026168346


인덱스가 어디서부턴가 틀렸다. 

# 문서 유사도 이용한 추천 시스템

# 문서로 재변환

In [ ]:
import pickle

#pickle 파일 불러오기
with open( "/content/drive/MyDrive/data/문서요약 텍스트/Training/index_to_word.pkl", "rb" ) as file:
    index_to_word = pickle.load(file)

print(index_to_word)

{1: '지역', 2: '사업', 3: '지원', 4: '위해', 5: '통해', 6: '기자', 7: '지난', 8: '대한', 9: '기업', 10: '면서', 11: '밝혔', 12: '계획', 13: '한국', 14: '위한', 15: '시장', 16: '진행', 17: '교육', 18: '도록', 19: '추진', 20: '운영', 21: '다는', 22: '다고', 23: '대표', 24: '올해', 25: '산업', 26: '관련', 27: '문화', 28: '개발', 29: '예정', 30: '도시', 31: '지난해', 32: '센터', 33: '경제', 34: '가능', 35: '정부', 36: '제공', 37: '사회', 38: '대상', 39: '참여', 40: '관계자', 41: '기술', 42: '보다', 43: '다양', 44: '의원', 45: '시민', 46: '라고', 47: '시설', 48: '기관', 49: '이날', 50: '주민', 51: '서울', 52: '관리', 53: '조사', 54: '활동', 55: '경우', 56: '서비스', 57: '정책', 58: '전국', 59: '라는', 60: '특히', 61: '대해', 62: '학교', 63: '환경', 64: '경기', 65: '현재', 66: '따르', 67: '행사', 68: '필요', 69: '마련', 70: '라며', 71: '선정', 72: '광주', 73: '대회', 74: '안전', 75: '투자', 76: '위원회', 77: '확대', 78: '평가', 79: '결과', 80: '학생', 81: '조성', 82: '최근', 83: '문제', 84: '이라고', 85: '국내', 86: '규모', 87: '이후', 88: '기준', 89: '해야', 90: '시작', 91: '국가', 92: '세계', 93: '개선', 94: '상황', 95: '연구', 96: '한편', 97: '분야', 98: '강화', 99: '일본', 100: '금융', 10

In [ ]:
decoded_text = []

for index in now_encoded_text_np[100] :
  decoded_text.append(index_to_word[index])

print(now_encoded_text_np[100])
print(decoded_text)

[591, 3100, 547, 1882, 89, 66724, 103, 1365, 19, 591, 3100, 547, 1882, 89, 66724, 103, 1365, 19, 19934, 6, 159, 117, 951, 1668, 17, 32, 202, 1450, 449, 1365, 922, 130, 996, 9392, 6, 1450, 449, 1828, 779, 5, 117, 1450, 1873, 1033, 30, 110, 21, 786, 367, 870, 2370, 3033, 310, 4069, 10, 117, 1450, 449, 1955, 547, 2, 5725, 591, 3100, 117, 260, 786, 17702, 117, 951, 1668, 17, 32, 1450, 449, 1365, 59, 350, 922, 591, 3100, 1, 1274, 5487, 794, 3097, 399, 143, 70, 66724, 2267, 1955, 103, 749, 2857, 1041, 14240, 310, 2965, 49, 4393, 119598, 591, 3100, 117, 260, 1015, 23507, 1043, 786, 14215, 817, 764, 9, 348, 8068, 1, 511, 4828, 822, 94, 1450, 449, 1955, 2370, 3033, 267, 149451, 117, 4988, 197, 1464, 612, 764, 45200, 2595, 1450, 449, 59, 536, 24688, 70, 2493, 870, 1114, 807, 3864, 441, 7079, 1575, 1837, 716, 471, 6526, 1639, 870, 25851, 1016, 9305, 84, 247, 159, 117, 951, 1668, 17, 32, 202, 1450, 449, 1365, 922, 130, 996, 9392, 6, 60, 922, 1450, 449, 1955, 872, 409, 114, 1736, 481, 2855, 114, 29

# 추가로 할 것
* preprocess에서 형태소 분석 > 자모분리
* TF-IDF 희소표현 > 밀집표현 word2vec ?